### Selected Word Group In Youtube Sentence

In [1]:
import os
import re
import pandas as pd
import numpy as np
import glob
from functools import reduce
from pathlib import Path
import shutil
from itertools import islice

In [2]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "Arabic"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# native word select for Part 2 
word_start = 0  # 0  # native word start index
word_end = 200 # 28  # native word end index

# source path
source_path = "/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Deployment2/Result/Deploy2 Result Manuel/Turkish"

# parameter
file = "Turkish_200_Word_Twogram_With_28_And_172_Word_Pair_In_Threegram_Selected"  # without xlsx extention
sheets = "selected_result"  # 2 gram target, 2 gram hybrid
file_ext = "Threegram_With_28_And_172_Word_Pair_Selected"
first_column = 0  # start column location number
end_column = 1
time_shift = 0.6  # 0.6
sample_num = 6  # 6

# youtube sentence condition
sent_time_start = False  # True, False for activating start time condition
sent_start_time = 700  # sn
random_sample = True  # True, False

In [3]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Deployment2/Result/7-Selected Word Group In Youtube Sentence/{lang_folder.capitalize()}"

Path(path).mkdir(parents=True, exist_ok=True)

In [4]:
def word_group_youtube(df, search_list, target_column, sample_num):
    '''
    word_group_youtube(df_youtube_sentence, search_list, "sentence", 6)
    ''' 
    df_search_result = pd.DataFrame()
    for j in search_list:
        try:
            df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)].sample(sample_num)
        except:
            df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)].head(sample_num)
        #df_result = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)]  # sentence length part
        #df_result.sort_values(f"{target_column}",key=lambda x:x.str.len(), inplace=True)
        #df_select = df_result.head(sample_num)
        df_select.insert(0,"search_string",j)
        df_search_result = pd.concat([df_search_result,df_select], axis=0)
    df_search_result.reset_index(inplace=True, drop=True)
    
    return df_search_result

In [5]:
def word_group_time_loc(df, search, start_sent, end_sent, sent, sent_video_id):
    '''
    word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
    '''
    df.reset_index(drop=True, inplace=True)
    word_time_loc_list = []
    for i in range(len(df)):
        word = df.loc[i,f"{search}"]
        start_time = df.loc[i,f"{start_sent}"]
        end_time = df.loc[i,f"{end_sent}"]
        sentence = df.loc[i,f"{sent}"]
        video_id = df.loc[i,f"{sent_video_id}"]
        time_length = end_time-start_time
        sentence_length = len(sentence)
        time_length_ratio = time_length/sentence_length
        loc_list = []
        for j in re.finditer(fr"(?:\s|^){word}(?:\s|$)", sentence, re.IGNORECASE|re.UNICODE):
            loc_list.append(j)
            start = loc_list[0].start()
            end = loc_list[0].end()
            start_loc = start_time+(start*time_length_ratio)
            end_loc = start_time+(end*time_length_ratio)
        word_time_loc_list.append([word,start_loc,end_loc,sentence,video_id])
    df_word_time_loc = pd.DataFrame(word_time_loc_list, columns=[f"{search}",f"{start_sent}",f"{end_sent}",f"{sent}",f"{sent_video_id}"])

    return df_word_time_loc

In [6]:
def word_count_result(df, column_list, set_condition=False): # df is dataframe, column_list is list value
    '''
    word_count_bool(df, column_list): df columns word count for word frequency\n
    df is dataframe, column_list is list value\n
    word_count_bool(df, ["word","twogram"]):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        if set_condition:
            var_list = set(df[f"{i}"].dropna().tolist())
        else:
            var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    df_word_count.sort_values("word_count", ascending=False, inplace=True)
    df_word_count.reset_index(inplace=True, drop=True)
    
    return  df_word_count

In [7]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

In [ ]:
df_word_select = df_word_all.iloc[word_start:word_end,]
df_word_select

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
195,saat,399989
196,onunla,399330
197,yapıyorsun,398274
198,neler,397377


In [ ]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{lang_folder.capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,00:00:00.294,00:00:03.294,bu dizinin betimlemesi staff fi lm tarafından,fI1BI4d5KFU
1,00:00:03.375,00:00:06.375,sesli betimleme derneğine yaptırılmıştır,fI1BI4d5KFU
2,00:00:06.462,00:00:09.462,wwwsebederorg,fI1BI4d5KFU
3,00:00:10.267,00:00:11.394,mumu,fI1BI4d5KFU
4,00:00:11.835,00:00:14.061,bak şu an sinir katsayım hat safhada,fI1BI4d5KFU
...,...,...,...,...
3934202,00:10:00.240,00:10:07.818,videoyu bitirmeden şunu da söylemek isterim ki...,dRssR_apbR0
3934203,00:10:07.994,00:10:13.534,bu nedenle ulaşımda insanların yarısına yakını...,dRssR_apbR0
3934204,00:10:14.745,00:10:17.924,danimarkayla alakalı şimdilik söyleyeceklerim ...,dRssR_apbR0
3934205,00:10:18.047,00:10:22.398,video hoşunuza gittiyse beğenmeyi ve kanalıma ...,dRssR_apbR0


In [ ]:
df_youtube_sentence['start_time'] = pd.to_timedelta(df_youtube_sentence['start_time']) # data type converted timedelta for second 
df_youtube_sentence['end_time'] = pd.to_timedelta(df_youtube_sentence['end_time'])

In [ ]:
df_youtube_sentence['start_time'] = df_youtube_sentence['start_time'].apply(lambda x: x.total_seconds()) # convert seconds
df_youtube_sentence['end_time'] = df_youtube_sentence['end_time'].apply(lambda x: x.total_seconds())
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,0.294,3.294,bu dizinin betimlemesi staff fi lm tarafından,fI1BI4d5KFU
1,3.375,6.375,sesli betimleme derneğine yaptırılmıştır,fI1BI4d5KFU
2,6.462,9.462,wwwsebederorg,fI1BI4d5KFU
3,10.267,11.394,mumu,fI1BI4d5KFU
4,11.835,14.061,bak şu an sinir katsayım hat safhada,fI1BI4d5KFU
...,...,...,...,...
3934202,600.240,607.818,videoyu bitirmeden şunu da söylemek isterim ki...,dRssR_apbR0
3934203,607.994,613.534,bu nedenle ulaşımda insanların yarısına yakını...,dRssR_apbR0
3934204,614.745,617.924,danimarkayla alakalı şimdilik söyleyeceklerim ...,dRssR_apbR0
3934205,618.047,622.398,video hoşunuza gittiyse beğenmeyi ve kanalıma ...,dRssR_apbR0


In [ ]:
if sent_time_start:
    df_youtube_select = df_youtube_sentence[df_youtube_sentence["start_time"] <= sent_start_time]
    df_youtube_select.reset_index(drop=True, inplace=True)
else:
    df_youtube_select = df_youtube_sentence

if random_sample:
    df_youtube_select = df_youtube_select.sample(len(df_youtube_select))
    df_youtube_select.reset_index(drop=True, inplace=True)
else:
    pass

df_youtube_select

,start_time,end_time,sentence,video_id
0,483.133,486.951,izin vermeyin sağlıklı kalın sağlıcakla kalın,y00__OTD4Rg
1,768.447,773.047,kim inanarak ve mükafatını allahtan bekleyerek,G82kzhYUINo
2,512.444,516.494,ancak cemal sıfatı olduğu gibi celali de vardır,PJm1DFTUxoQ
3,399.980,400.980,ayrı ayrı seçiyorsunuz,akzpB69pSwk
4,6609.965,6612.535,artık kızımıza layık bir düğün yapıyoruz hayriye,ztZMp7dpRto
...,...,...,...,...
3934202,5677.460,5679.860,inandıramadılar ama bir yerden açık verdiler işte,oCPRsBiXlf4
3934203,934.832,937.033,ha o işte o tamam kapat sen kapat,480nF81vFI0
3934204,5906.780,5908.920,ömür hanımın yakın bir akrabası,dXYd77Gevj8
3934205,1278.072,1279.199,haklısın,XWpMMIj8REs


In [ ]:
df_word_group_select = pd.read_excel(f"{source_path}/{file}.xlsx", sheet_name= f"{sheets}")
df_word_group_select

,twogram
0,biz de
1,yok artık
2,söyle bana
3,bir şeyler
4,bu doğru
...,...
57,evet tabii
58,daha fazla
59,oh evet
60,ne zaman


In [ ]:
# Option 
# Skip if it doesn't need
df_remove = pd.read_excel(f"")
df_remove

In [ ]:
remove_videoid_list = list(set(df_remove["video_id"]))
#remove_videoid_list

In [ ]:
len(remove_videoid_list)

In [ ]:
# Option
df_youtube_select = df_youtube_select[~df_youtube_select["video_id"].isin(remove_videoid_list)]
df_youtube_select

In [ ]:
search_list = []
for i in range(len(df_word_group_select)):
    for j in df_word_group_select.columns[first_column:end_column]:
        string = df_word_group_select.loc[i,j]
        
        if pd.isnull(string) == False and string != 'nan':
            search_list.append(string.strip())
            search_list = list(set(search_list))  # for unique value
        else:
            pass

In [ ]:
len(search_list)

62

In [ ]:
df_search_result = word_group_youtube(df_youtube_select, search_list, "sentence", sample_num)
df_search_result

,search_string,start_time,end_time,sentence,video_id
0,bir şeyler,7397.320,7398.580,ben gidip bir şeyler alayım,Wxk7wlMV04I
1,bir şeyler,4817.235,4819.565,toprak da olsun diye bir şeyler söylemiş,AuFFdIhuqs0
2,bir şeyler,4529.680,4534.120,fırat bey benden isteseydiniz hemen size yiyec...,cQbynkEPQqs
3,bir şeyler,1494.020,1497.120,bir şeyler duydum onun hayatıyla ilgili,t_HAPmrYZrY
4,bir şeyler,5501.732,5503.093,hadi bir şeyler ye,pGS6GfPOG8s
...,...,...,...,...,...
364,bunun için,473.340,475.640,bunun için ihtiyaç duyduğu roketlere gelince,pdGlicIifJY
365,bunun için,1834.950,1839.741,i zmirin diğer ucuna gidiyorduk sırf bunun için,_Ya18fCiA0k
366,bunun için,40.207,45.000,gezinmesini istiyoruz ve bunun için en güzel ö...,MYLvmIkVFko
367,bunun için,3650.173,3655.173,olmaktan nasıl kaçınıyor bunun için ne yapıyor,uQsxPYqkFfw


In [ ]:
df_word_group_time_loc_result = word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc_result

,search_string,start_time,end_time,sentence,video_id
0,bir şeyler,7397.740000,7398.300000,ben gidip bir şeyler alayım,Wxk7wlMV04I
1,bir şeyler,4818.400000,4819.099000,toprak da olsun diye bir şeyler söylemiş,AuFFdIhuqs0
2,bir şeyler,4532.681690,4533.432113,fırat bey benden isteseydiniz hemen size yiyec...,cQbynkEPQqs
3,bir şeyler,1494.020000,1494.894359,bir şeyler duydum onun hayatıyla ilgili,t_HAPmrYZrY
4,bir şeyler,5502.034444,5502.941778,hadi bir şeyler ye,pGS6GfPOG8s
...,...,...,...,...,...
364,bunun için,473.340000,473.915000,bunun için ihtiyaç duyduğu roketlere gelince,pdGlicIifJY
365,bunun için,1838.619702,1839.741000,i zmirin diğer ucuna gidiyorduk sırf bunun için,_Ya18fCiA0k
366,bunun için,41.720579,42.477368,gezinmesini istiyoruz ve bunun için en güzel ö...,MYLvmIkVFko
367,bunun için,3652.781696,3654.086043,olmaktan nasıl kaçınıyor bunun için ne yapıyor,uQsxPYqkFfw


In [ ]:
df_word_group_time_loc_result.start_time = df_word_group_time_loc_result.start_time.apply(lambda x: (x-time_shift))
df_word_group_time_loc_result.end_time = df_word_group_time_loc_result.end_time.apply(lambda x: (x+time_shift))
df_word_group_time_loc_result

,search_string,start_time,end_time,sentence,video_id
0,bir şeyler,7397.140000,7398.900000,ben gidip bir şeyler alayım,Wxk7wlMV04I
1,bir şeyler,4817.800000,4819.699000,toprak da olsun diye bir şeyler söylemiş,AuFFdIhuqs0
2,bir şeyler,4532.081690,4534.032113,fırat bey benden isteseydiniz hemen size yiyec...,cQbynkEPQqs
3,bir şeyler,1493.420000,1495.494359,bir şeyler duydum onun hayatıyla ilgili,t_HAPmrYZrY
4,bir şeyler,5501.434444,5503.541778,hadi bir şeyler ye,pGS6GfPOG8s
...,...,...,...,...,...
364,bunun için,472.740000,474.515000,bunun için ihtiyaç duyduğu roketlere gelince,pdGlicIifJY
365,bunun için,1838.019702,1840.341000,i zmirin diğer ucuna gidiyorduk sırf bunun için,_Ya18fCiA0k
366,bunun için,41.120579,43.077368,gezinmesini istiyoruz ve bunun için en güzel ö...,MYLvmIkVFko
367,bunun için,3652.181696,3654.686043,olmaktan nasıl kaçınıyor bunun için ne yapıyor,uQsxPYqkFfw


In [ ]:
df_word_group_time_loc_result.start_time = df_word_group_time_loc_result.start_time.apply(lambda x: round(x))
df_word_group_time_loc_result.end_time = df_word_group_time_loc_result.end_time.apply(lambda x: round(x))
df_word_group_time_loc_result 

,search_string,start_time,end_time,sentence,video_id
0,bir şeyler,7397,7399,ben gidip bir şeyler alayım,Wxk7wlMV04I
1,bir şeyler,4818,4820,toprak da olsun diye bir şeyler söylemiş,AuFFdIhuqs0
2,bir şeyler,4532,4534,fırat bey benden isteseydiniz hemen size yiyec...,cQbynkEPQqs
3,bir şeyler,1493,1495,bir şeyler duydum onun hayatıyla ilgili,t_HAPmrYZrY
4,bir şeyler,5501,5504,hadi bir şeyler ye,pGS6GfPOG8s
...,...,...,...,...,...
364,bunun için,473,475,bunun için ihtiyaç duyduğu roketlere gelince,pdGlicIifJY
365,bunun için,1838,1840,i zmirin diğer ucuna gidiyorduk sırf bunun için,_Ya18fCiA0k
366,bunun için,41,43,gezinmesini istiyoruz ve bunun için en güzel ö...,MYLvmIkVFko
367,bunun için,3652,3655,olmaktan nasıl kaçınıyor bunun için ne yapıyor,uQsxPYqkFfw


In [ ]:
df_word_group_time_loc_result["video_url"] = "https://www.youtube.com/watch?v="+df_word_group_time_loc_result['video_id'].map(str)+"&t="+df_word_group_time_loc_result['start_time'].map(str)+"s"
df_word_group_time_loc_result

,search_string,start_time,end_time,sentence,video_id,video_url
0,bir şeyler,7397,7399,ben gidip bir şeyler alayım,Wxk7wlMV04I,https://www.youtube.com/watch?v=Wxk7wlMV04I&t=...
1,bir şeyler,4818,4820,toprak da olsun diye bir şeyler söylemiş,AuFFdIhuqs0,https://www.youtube.com/watch?v=AuFFdIhuqs0&t=...
2,bir şeyler,4532,4534,fırat bey benden isteseydiniz hemen size yiyec...,cQbynkEPQqs,https://www.youtube.com/watch?v=cQbynkEPQqs&t=...
3,bir şeyler,1493,1495,bir şeyler duydum onun hayatıyla ilgili,t_HAPmrYZrY,https://www.youtube.com/watch?v=t_HAPmrYZrY&t=...
4,bir şeyler,5501,5504,hadi bir şeyler ye,pGS6GfPOG8s,https://www.youtube.com/watch?v=pGS6GfPOG8s&t=...
...,...,...,...,...,...,...
364,bunun için,473,475,bunun için ihtiyaç duyduğu roketlere gelince,pdGlicIifJY,https://www.youtube.com/watch?v=pdGlicIifJY&t=...
365,bunun için,1838,1840,i zmirin diğer ucuna gidiyorduk sırf bunun için,_Ya18fCiA0k,https://www.youtube.com/watch?v=_Ya18fCiA0k&t=...
366,bunun için,41,43,gezinmesini istiyoruz ve bunun için en güzel ö...,MYLvmIkVFko,https://www.youtube.com/watch?v=MYLvmIkVFko&t=41s
367,bunun için,3652,3655,olmaktan nasıl kaçınıyor bunun için ne yapıyor,uQsxPYqkFfw,https://www.youtube.com/watch?v=uQsxPYqkFfw&t=...


In [ ]:
search_result_num = df_word_group_time_loc_result["search_string"].nunique()
search_result_num

62

In [ ]:
search_result_set = set(df_word_group_time_loc_result["search_string"])
search_all_set = set(search_list)

In [ ]:
search_all_set.difference(search_result_set)

set()

In [ ]:
df_result_word_manuel_count = word_count_result(df_word_group_time_loc_result, df_word_group_time_loc_result.columns[0:1], set_condition=True)
df_result_word_manuel_count

,word,word_count
0,değil,4
1,bu,4
2,de,4
3,hayır,4
4,daha,3
...,...,...
72,sorun,1
73,kadar,1
74,nerede,1
75,neden,1


In [ ]:
# # Option merge
# df_word_count_merge = pd.merge(df_result_word_manuel_count, df_word_select[["word"]], how="inner", on="word")
# df_word_count_merge.drop_duplicates(inplace=True)
# df_word_count_merge.sort_values(by="word_count", ascending=False, inplace=True)
# df_word_count_merge.reset_index(drop=True, inplace=True)
# df_word_count_merge

In [ ]:
set_word = set(df_word_select["word"])
#set_count = set(df_word_count_merge["word"])  # Option merge
set_count = set(df_result_word_manuel_count["word"])

In [ ]:
set_word.difference(set_count)

{'adam',
 'al',
 'anne',
 'asla',
 'aslında',
 'baba',
 'bak',
 'bakalım',
 'başka',
 'bence',
 'benimle',
 'bile',
 'biliyor',
 'bilmiyorum',
 'biri',
 'birkaç',
 'birlikte',
 'bize',
 'bizi',
 'bugün',
 'buna',
 'bütün',
 'büyük',
 'dakika',
 'devam',
 'diye',
 'dostum',
 'dur',
 'ederim',
 'en',
 'et',
 'eğer',
 'gel',
 'geldi',
 'geliyor',
 'gerek',
 'geri',
 'gerçek',
 'git',
 'göre',
 'gün',
 'güzel',
 'hadi',
 'hakkında',
 'hala',
 'harika',
 'hemen',
 'her',
 'herkes',
 'hey',
 'hiç',
 'iki',
 'ile',
 'ilk',
 'ister',
 'istiyorsun',
 'istiyorum',
 'izin',
 'içinde',
 'iş',
 'işte',
 'kendi',
 'ki',
 'küçük',
 'kız',
 'lazım',
 'merhaba',
 'musun',
 'mü',
 'mısın',
 'nasıl',
 'neler',
 'ol',
 'olabilir',
 'olacak',
 'olan',
 'olarak',
 'oldu',
 'olduğunu',
 'olmak',
 'olsun',
 'olur',
 'oluyor',
 'ona',
 'onları',
 'onun',
 'onunla',
 'orada',
 'para',
 'pekala',
 'peki',
 'saat',
 'sadece',
 'sanırım',
 'selam',
 'senin',
 'seninle',
 'siz',
 'size',
 'sizi',
 'son',
 'tam',
 '

In [ ]:
df_word_group_time_loc_result.to_excel(f"{lang_folder.capitalize()}_{search_result_num}_{file_ext}_\
{sample_num}_Youtube_{time_shift}s_Timeshift_Result.xlsx", index=False) 

In [ ]:
# df_result_word_manuel_count.to_excel(f"{lang_folder.capitalize()}_{search_result_num}_{file_ext}_\
# {sample_num}_Youtube_{time_shift}s_Timeshift_Result_Word_Count.xlsx", index=False) 

#### Copy Move And Delete

In [ ]:
output_file = glob.glob(f"{lang_folder.capitalize()}_*_{sample_num}_Youtube_{time_shift}s_Timeshift_Result*.xlsx")
output_file

['Turkish_62_Twogram_With_28_And_172_Word_Pair_Selected_6_Youtube_0.6s_Timeshift_Result.xlsx']

In [ ]:
for y in output_file:
    source = y # source directory
    destination = path
    shutil.copy2(source, destination)

In [ ]:
for z in output_file:
    try:
        os.remove(z)
    except:
        pass

#### Temp

In [ ]:
set_result = set(df_word_group_time_loc_result["search_string"].unique())
set_selected = set(search_list)

In [ ]:
df_not_in_result = pd.DataFrame(set_selected.difference(set_result), columns=["not_in_youtube_result"])
df_not_in_result

In [ ]:
not_in_result_num = len(df_not_in_result)
not_in_result_num

In [ ]:
#df_not_in_result.to_excel(f"{lang_folder.capitalize()}_{not_in_result_num}_{file_ext}_\
#Not_In_{sample_num}_Youtube_{time_shift}s_Timeshift_Result.xlsx", index=False) 

In [ ]:
search_list2 = df_not_in_result["not_in_youtube_result"].to_list()

In [ ]:
df_search_result2 = word_group_youtube(df_youtube_sentence, search_list2, "sentence", sample_num)
df_search_result2

In [ ]:
search_result2_num = df_search_result2["search_string"].nunique()
search_result2_num

In [ ]:
df_word_group_time_loc_result2 = word_group_time_loc(df_search_result2, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc_result2

In [ ]:
df_word_group_time_loc_result2.start_time = df_word_group_time_loc_result2.start_time.apply(lambda x: (x-time_shift))
df_word_group_time_loc_result2.end_time = df_word_group_time_loc_result2.end_time.apply(lambda x: (x+time_shift))
df_word_group_time_loc_result2

In [ ]:
df_word_group_time_loc_result2["video_url"] = "https://www.youtube.com/watch?v="+df_word_group_time_loc_result2['video_id'].map(str)+"&t="+df_word_group_time_loc_result2['start_time'].map(str)+"s"
df_word_group_time_loc_result2

In [ ]:
search_result_num2 = df_word_group_time_loc_result2["search_string"].nunique()
search_result_num2

In [ ]:
df_word_group_time_loc_result2.to_excel(f"{lang_folder.capitalize()}_{search_result_num2}_{file_ext}_\
{sample_num}_Youtube_{time_shift}s_Timeshift_Result2.xlsx", index=False) 